## My Sportify - Music Recommendation System -

#### 2019-01-09 Wed

In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime
import math
import gc

from subprocess import check_output
print(check_output(["ls", "../KKBox"]).decode("utf8"))

Sportify.pdf
members.csv
members.csv.7z
sample_submission.csv
sample_submission.csv.7z
song_extra_info.csv
song_extra_info.csv.7z
songs.csv
songs.csv.7z
sportify.ipynb
test.csv
test.csv.7z
train.csv
train.csv.7z



In [ ]:
#CSVファイルからデータを読み込みデータフレームを作成

print("Loading data...")
path = '../KKBox/'
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")
songs = pd.read_csv(path + "songs.csv")
members = pd.read_csv(path + "members.csv")
song_extra_info = pd.read_csv(path + "song_extra_info.csv")
print("Done Loading...")

Loading data...


In [118]:
#Checking each DataFrame

print("-----train-----")
print(train.shape, train.columns)
print("\n-----test-----")
print(test.shape, test.columns)
print("\n-----songs-----")
print(songs.shape, songs.columns)
print("\n-----members-----")
print(members.shape, members.columns)
print("\n-----song_extra_info-----")
print(song_extra_info.shape, song_extra_info.columns)

-----train-----
(7377418, 6) Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target'],
      dtype='object')

-----test-----
(2556790, 6) Index(['id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type'],
      dtype='object')

-----songs-----
(2296320, 7) Index(['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language'],
      dtype='object')

-----members-----
(34403, 7) Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date'],
      dtype='object')

-----song_extra_info-----
(2295971, 3) Index(['song_id', 'name', 'isrc'], dtype='object')


In [119]:
train.drop(labels=["source_system_tab", "source_screen_name", "source_type"], axis=1, inplace=True)
train.columns = ["UserID", "SongID", "Target"]
print(train.isnull().sum())
train.head(3)

UserID    0
SongID    0
Target    0
dtype: int64


,UserID,SongID,Target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1


In [120]:
songs = songs.drop(labels=["composer", "lyricist"], axis=1)
songs = songs[["song_id", "language", "artist_name", "genre_ids", "song_length"]]
songs.columns = ["SongID", "Language", "Artist", "Category", "Length"]
print(songs.isnull().sum())
songs.head()

TypeError: 'NoneType' object is not subscriptable

In [121]:
members_new = members.drop(labels=["registered_via", "registration_init_time", "expiration_date"], axis=1)
members = members_new[["msno", "bd", "gender", "city"]]
members.columns = ["UserID", "Age", "Sex", "City"]
members.head(3)

,UserID,Age,Sex,City
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,0,NaN,1
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,0,NaN,1
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,0,NaN,1


In [112]:
members.city.nunique()

21

In [114]:
members.head(3)

AttributeError: 'NoneType' object has no attribute 'head'

In [36]:
members.head(3)

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712


In [42]:
song_extra_info.head(3)

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
